# Personalised recommendations to increase AOV of Instacart loyalists

---
## Part 1: Data merging

In this notebook, I will perform file size reduction and merging of datasets which are then exported for EDA in the next notebook.

The following are the relational datasets provided by Instacart:
   
`orders` (3.4m rows, 206k users):
* `order_id`: order identifier
* `user_id`: customer identifier
* `eval_set`: which evaluation set this order belongs in (see `SET` described below)
* `order_number`: the order sequence number for this user (1 = first, n = nth)
* `order_dow`: the day of the week the order was placed on
* `order_hour_of_day`: the hour of the day the order was placed on
* `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

`products` (50k rows):
* `product_id`: product identifier
* `product_name`: name of the product
* `aisle_id`: foreign key
* `department_id`: foreign key

`aisles` (134 rows):
* `aisle_id`: aisle identifier
* `aisle`: the name of the aisle

`departments` (21 rows):
* `department_id`: department identifier
* `department`: the name of the department

`order_products__train` (1.4m+ rows) and `order_products__prior` (30m+ rows):
* `order_id`: foreign key
* `product_id`: foreign key
* `add_to_cart_order`: order in which each product was added to cart
* `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

where `SET` is one of the four following evaluation sets (`eval_set` in `orders`):
* `"prior"`: all users' orders excluding their last order (~3.2m orders)
* `"train"`: the last order of a subset of customers (~131k orders)
* `"test"`: the last order of the remaining customers without order details (~75k orders)

This dataset was originally meant for a Kaggle competition where users were meant to predict items in the user's next basket. Hence, the last order details for a subset of customers were hidden (corresponding to rows where `eval_set` = `test`) so that predictions could be made on which items were in that order. 

However, since the problem statement I am addressing is not the prediction of next-basket items, I will remove all rows where `eval_set` = `test` so that I will have the full details of all orders in the dataset. I will also join `order_products__train` and `order_products__prior` together as `order_products` since I do not need the last orders to be kept separate from all prior orders.

Finally, I will be consolidating our dataframes into two main dataframes – one containing product information only and the other containing details of all orders their associated products. 

---

### Load datasets

In [1]:
import pickle
import pandas as pd

In [2]:
orders = pd.read_csv('../datasets/orders.csv', 
                     dtype= {
                                'order_id': 'uint32',
                                'user_id': 'uint32',
                                'order_number': 'uint8',
                                'order_dow': 'uint8',
                                'order_hour_of_day': 'uint8',
                                'days_since_prior_order': 'float32'    
                            })

order_products_train = pd.read_csv('../datasets/order_products__train.csv', 
                                   dtype={
                                            'order_id': 'uint32',
                                            'product_id': 'uint16',
                                            'add_to_cart_order': 'uint8',
                                            'reordered': 'uint8'   
                                        })

order_products_prior = pd.read_csv('../datasets/order_products__prior.csv', 
                                   dtype={
                                            'order_id': 'uint32',
                                            'product_id': 'uint16',
                                            'add_to_cart_order': 'uint8',
                                            'reordered': 'uint8'   
                                         })

products = pd.read_csv('../datasets/products.csv', 
                       dtype={
                                'product_id': 'uint16',
                                'aisle_id': 'uint8',
                                'department_id': 'uint8'
                            })

depts = pd.read_csv('../datasets/departments.csv', 
                    dtype={
                            'department_id': 'uint8',
                            'department': 'category'
                          })

aisles = pd.read_csv('../datasets/aisles.csv', 
                     dtype={
                            'aisle_id': 'uint8',
                            'aisle': 'category'
                            })

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [22]:
order_products_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [5]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
depts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [8]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


### Merge dataframes

#### Merge `order_products_train` and `order_products_prior`

In [9]:
# combine `order_products_train`, `order_products_prior`
order_products = pd.concat([order_products_train, order_products_prior])

In [10]:
order_products

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


#### Delete rows for eval_set=test from `orders`

In [11]:
# drop rows for eval_set=test because we are missing the corresponding order details in the `order_products` dataset
orders = orders[orders['eval_set'] != 'test']

In [12]:
# drop the `eval_set` column since it's not needed anymore
del orders['eval_set']

#### Merge `orders` and `order_products`

In [13]:
orders_df = pd.merge(orders, order_products, on='order_id', how='inner')
orders_df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,196,1,0
1,2539329,1,1,2,8,NaN,14084,2,0
2,2539329,1,1,2,8,NaN,12427,3,0
3,2539329,1,1,2,8,NaN,26088,4,0
4,2539329,1,1,2,8,NaN,26405,5,0


#### Merge `products`, `departments`, and `aisles` into a single dataframe

In [14]:
# merge `products` and `departments` first
products_df = pd.merge(products, depts, on='department_id')

In [15]:
# merge with `aisles`
products_df = pd.merge(products_df, aisles, on='aisle_id')
products_df.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,snacks,cookies cakes
2,102,Danish Butter Cookies,61,19,snacks,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,snacks,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,snacks,cookies cakes


#### Finally, merge the Products and Orders dataframes together

In [16]:
df = pd.merge(orders_df, products_df, on='product_id', how='inner')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 33819105
Data columns (total 14 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                uint32  
 1   user_id                 uint32  
 2   order_number            uint8   
 3   order_dow               uint8   
 4   order_hour_of_day       uint8   
 5   days_since_prior_order  float32 
 6   product_id              uint16  
 7   add_to_cart_order       uint8   
 8   reordered               uint8   
 9   product_name            object  
 10  aisle_id                uint8   
 11  department_id           uint8   
 12  department              category
 13  aisle                   category
dtypes: category(2), float32(1), object(1), uint16(1), uint32(2), uint8(7)
memory usage: 1.3+ GB


In [18]:
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,beverages,soft drinks
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,beverages,soft drinks
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,beverages,soft drinks
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,beverages,soft drinks
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,beverages,soft drinks


In [19]:
# delete the aisle_id and department_id columns since they are redundant
df.drop(columns=['aisle_id', 'department_id'], inplace=True)

### Pickle files for subsequent use

In [20]:
# created files
# orders_df.to_pickle("../datasets/orders_df.pkl") # redundant file
products_df.to_pickle("../datasets/products_df.pkl")
df.to_pickle('../datasets/full_combined_df.pkl')